<a href="https://colab.research.google.com/github/Eliasamuneke/Quickstart_lab/blob/main/Working_version_of_bagging_n_pasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bagging and Pasting

#### Part of the [Inquiryum Machine Learning Fundamentals Course](http://inquiryum.com/machine-learning/)

![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/divider.png)

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/bagging..png)


Now we are about to embark on our journey from simple decision trees to algorithms that use decision trees as components. The path goes like this:


![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/dbxg.png)

The use of decision trees began in the 1980s and XGBoost was introduced in 2016. Throughout the next few notebooks we will explore this progression of algorithms.  

### A collective of classifiers

To gain an intuition on how this works, let's look at how our confidence might increase when more people tell us something. Whether it is multiple doctors giving us the same diagnosis or something as simple as ...

#### The Mary Spender example

Let's say one of your friends mentions over lunch that you would love a particular musical artist on YouTube, say Mary Spender, who you never heard before. 

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/MarySpender2.png)

What is the chance that you will actually like Mary Spender's music? Maybe slighly better than chance? Let's say you think there is a 60% chance you will like her. You will file away the recommendation but you are not going to rush home and watch a YouTube video.  Now, in addition to the lunch friend's recommendation,  an old music school friend, now living in Austin messages you saying you should check out Mary Spender and the friend predicts you will absolutely love her. Then a week later, while talking with an old bandmate over the phone, that bandmate, again, recommends Mary Spender. Over the course of less than 10 days, three of your friends independently (because they don't know one another) recommend Mary Spender. Now what is the likelihood of you liking Mary Spender? I am guessing you think that now it is higher than 60%. Maybe now you think it is 90% likely you will like her. It is the aggregate of these 3 people's opinions (3 classifiers) that ups the accuracy of the prediction.

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/spender22.png)


This is similar to how bagging works. One aggregates the votes of a number of classifiers and the vote of that ensemble of classifiers is more accurate than that of a single classifier. Even if the accuracy of each component classifier is low (known as a weak classifier), the ensemble can be a strong (high accuracy) classifier. Of course there are some caveats. 

Back to the Mary Spender example. Suppose one of your friends went to a Mary Spender concert and then later in the week met with four other of your friends and mentioned that she thought you would love Mary Spender's music. Then, over the course of a week all those friends recommended Mary Spender to you. In this case the recommendations are not that independent---all are based on one person's opinion. Thus, the accuracy would not be as great as in the example above. Similarly, if you made 10 copies of the exact same classifier each trained on exactly the same data, the accuracy of the ensemble of clones would not be any better than the accuracy of a single copy. Moving away from Mary Spender and our musical tastes and back to machine learning, we can try to create independence among the classifier in 2 ways:

1. We can change the type of classifier. For example, we can use a k-Nearest Neighbor Classifier with Manhattan distance and a k of 5, a k-Nearest Neighbor Classifier with Euclidean distance and a k of 3, a decision tree classifier using entropy and a max depth of 5, and a decision tree classifier with using gini and no max depth specified. Hopefully, the accuracy of the ensemble of the four classifiers would be greater than that of a single classifier.
2. We can have an ensemble of the same classifier (for example, 10 decision tree classifiers with identical hyperparameters) but each classifier can get a different subset of the training data. The classifiers would thus build different models (differents 'rules') and, again, the accuracy of the ensemble should be greather than that of a single classifier. This is the approach we will take.

### Bagging and Pasting


In this Jupyter notebook, we are going to explore Bagging algorithms. Bagging algorithms come in a variety of 'flavors' including one called 'bagging' and one called 'pasting'.

But first an experiment on the what *with replacement* means. As you will see shortly, that term is the crucial difference between bagging and pasting.

### A small experiment
NOTE: The following code is just used for illustration and is nothing we will be using for machine learning. 

Consider a list of 5 red balls and 5 blue balls:

In [1]:
bag = ['red', 'red', 'red', 'red', 'red',
       'blue', 'blue', 'blue', 'blue', 'blue']

Suppose we want to pick 7 random balls from this list. Python offers two functions that will give us random elements from a list.One is called `choices` which selects a sample with replacement, which means that once a ball is selected it is put back in the bag so it has the potential to be selected again. Let's give it a try, and just because things are random let's do this 100 times:

In [2]:
import random
total = 0
for i in range(100):
    set = random.choices(bag, k=7)
    blue = set.count('blue')
    red = set.count('red')
    if blue > 5 or red > 5:
        print("%i blue and %i red" % (set.count('blue'), set.count('red')))
        total +=1
print("Balls selected exceeded balls in bag: %i" % (total))

6 blue and 1 red
1 blue and 6 red
6 blue and 1 red
1 blue and 6 red
6 blue and 1 red
6 blue and 1 red
7 blue and 0 red
6 blue and 1 red
1 blue and 6 red
1 blue and 6 red
Balls selected exceeded balls in bag: 10


*A reminder: please don't mindlessly execute the code. Look at it and understand it*

There are five blue balls. Since we are doing the selection with replacement there are times when we select more than 5 blue balls (or five red ones).  

When we print

```
Balls selected exceeded balls in bag: 13
```

it shows how many times that was the case.


When I ran this, 14 times out of 100 had more of one color ball than there were in the original bag. In fact, several times I ended up with all 7 of the balls blue, even though the original list had only 5 balls:

```
7 blue and 0 red
6 blue and 1 red
7 blue and 0 red
1 blue and 6 red
0 blue and 7 red
1 blue and 6 red
1 blue and 6 red
6 blue and 1 red
6 blue and 1 red
1 blue and 6 red
6 blue and 1 red
1 blue and 6 red
6 blue and 1 red
6 blue and 1 red
Balls selected exceeded balls in bag: 14
```
Again, this is called selecting with replacement (we put what we selected back in the set before selecting again). 

The other alternative is to select without replacement--once we select something we can't select it again. Python's `sample` does this:


In [3]:
import random
total = 0
for i in range(1000):
    set = random.sample(bag, k=7)
    blue = set.count('blue')
    red = set.count('red')
    if blue > 5 or red > 5:
        print("%i blue and %i red" % (set.count('blue'), set.count('red')))
        total +=1
print("Balls selected exceeded balls in bag: %i" % (total))

Balls selected exceeded balls in bag: 0


As you can see, the number of a specific colored ball that we select never exceeded the number of balls of that color in the original set.

Now back to bagging and pasting. In both approaches we are going to sample the training data. Let's say we want 70% of the training data in our sample. In bagging ([Breiman, 1996](https://link.springer.com/content/pdf/10.1007/BF00058655.pdf)), if we our training dataset is 1000 instances and we want 70% for a particular classifier, the algorithm will randomly select 700 out of the 1,000 **with replacement**. With pasting ([Breiman, 1998](https://link.springer.com/article/10.1023/A:1007563306331)), the selection is done **without replacement**. 

#### but wait, there is more ...

There are two other options. Instead of selecting a random subset of training data instances, we can select a random subset of columns (features). Let's say we have a dataset of 1,000 instances each with 100 features. When we select a random subset of columns, we still have 1,000 instances but now they have just a subset of the features. This is called Random Subspaces ([Ho, 1998](https://pdfs.semanticscholar.org/b41d/0fa5fdaadd47fc882d3db04277d03fb21832.pdf?_ga=2.196949164.1638238666.1596910000-1073138517.1596910000)).

Finally, we can train a classifier on both random subsets of instances and random subsets of features. This is known as Random Patches ([Louppe and Geurts, 2012](https://www.researchgate.net/publication/262212941_Ensembles_on_Random_Patches))

In summary, the four methods are:

* **bagging** - select a subset of data set instances using replacement
* **pasting** - select a subset of data set instances without replacement
* **Random Subspaces** - select a subset of features
* **Random Patches** - select both a subset of features and of instances

Let's see how this works!

First, let's grab the Wisconsin Cancer data we used before:

#### Wisconsin Cancer Dataset
![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/aimam.png)
image from Nvidia's [AI Improves Breast Cancer Diagnoses by Factoring Out False Positives](https://blogs.nvidia.com/blog/2018/02/01/making-mammography-more-meaningful/)

[A description of the Cancer Database](#Breast-Cancer-Database)

In this dataset we are trying to predict the diagnosis---either M (malignant) or B (benign).

Let's load the dataset and split it into training and testing sets

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

colNames = ['id', 'diagnosis', 'radiusAvg', 'textureAvg', 'perimeterAvg', 'areaAvg',
            'smoothnessAvg', 'compactnessAvg', 'concavityAvg', 'concavityPointsAvg',
            'symmetryAvg', 'FractalDimensionAvg', 'radiusSE', 'textureSE', 'perimeterSE',
            'areaSE','smoothnessSE', 'compactnessSE', 'concavitySE', 'concavityPointsSE',
            'symmetrySE', 'FractalDimensionSE', 'radiusWorst', 'textureWorst', 'perimeterWorst',
            'areaWorst', 'smoothnessWorst', 'compactnessWorst', 'concavityWorst', 'concavityPointsWorst',
            'symmetryWorst>', 'FractalDimensionWorst']
len(colNames)

data = pd.read_csv('https://raw.githubusercontent.com/zacharski/ml-class/master/data/wdbc.data', names=colNames)
data.set_index('id', inplace=True)

trainingdata, testdata = train_test_split(data, test_size = 0.2)
testdata

,diagnosis,radiusAvg,textureAvg,perimeterAvg,areaAvg,smoothnessAvg,compactnessAvg,concavityAvg,concavityPointsAvg,symmetryAvg,...,radiusWorst,textureWorst,perimeterWorst,areaWorst,smoothnessWorst,compactnessWorst,concavityWorst,concavityPointsWorst,symmetryWorst>,FractalDimensionWorst
id,,,,,,,,,,,,,,,,,,,,,
855563,M,10.95,21.35,71.90,371.1,0.12270,0.12180,0.10440,0.05669,0.1895,...,12.840,35.34,87.22,514.0,0.19090,0.26980,0.40230,0.14240,0.2964,0.09606
857343,B,11.76,21.60,74.72,427.9,0.08637,0.04966,0.01657,0.01115,0.1495,...,12.980,25.72,82.98,516.5,0.10850,0.08615,0.05523,0.03715,0.2433,0.06563
865432,B,14.50,10.89,94.28,640.7,0.11010,0.10990,0.08842,0.05778,0.1856,...,15.700,15.98,102.80,745.5,0.13130,0.17880,0.25600,0.12210,0.2889,0.08006
84799002,M,14.54,27.54,96.73,658.8,0.11390,0.15950,0.16390,0.07364,0.2303,...,17.460,37.13,124.10,943.2,0.16780,0.65770,0.70260,0.17120,0.4218,0.13410
9113538,M,17.60,23.33,119.00,980.5,0.09289,0.20040,0.21360,0.10020,0.1696,...,21.570,28.87,143.60,1437.0,0.12070,0.47850,0.51650,0.19960,0.2301,0.12240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875099,B,9.72,18.22,60.73,288.1,0.06950,0.02344,0.00000,0.00000,0.1653,...,9.968,20.83,62.25,303.8,0.07117,0.02729,0.00000,0.00000,0.1909,0.06559
88649001,M,19.55,28.77,133.60,1207.0,0.09260,0.20630,0.17840,0.11440,0.1893,...,25.050,36.27,178.60,1926.0,0.12810,0.53290,0.42510,0.19410,0.2818,0.10050
89511501,B,12.20,15.21,78.01,457.9,0.08673,0.06545,0.01994,0.01692,0.1638,...,13.750,21.38,91.11,583.1,0.12560,0.19280,0.11670,0.05556,0.2661,0.07961


Now divide up the data into the features and labels

In [5]:
colNames.remove('id')
colNames.remove('diagnosis')
trainingDataFeatures = trainingdata[colNames]
testDataFeatures = testdata[colNames]
trainingDataLabels = trainingdata['diagnosis']
testDataLabels = testdata['diagnosis']


Let's get a base accuracy using a single decision tree classifier:

In [6]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion='entropy')
clf.fit(trainingDataFeatures, trainingDataLabels)
predictions = clf.predict(testDataFeatures)

accuracy_score(testDataLabels, predictions)

0.9473684210526315

Now we will see if we can improve on that accuracy.

### Building a bagging classifier

Let's build a collective of 20 decision tree classifiers (`n_estimators`). Let's train each one with 100 random samples from our dataset (`max_samples`) with replacement (`bootstrap=True`). `n_jobs` means how many jobs to run in parallel. `n_jobs=-1` means use all available CPU cores.   

Just to reinforce the vocabulary we are learning, `n_estimators`, `max_samples`, `bootstrap` are among the **hyperparameters** of the bagging classifier.

In [7]:
from sklearn.ensemble import BaggingClassifier
clf = tree.DecisionTreeClassifier(criterion='entropy')

bagging_clf = BaggingClassifier(clf, n_estimators=20, max_samples=100, 
                                bootstrap=True, n_jobs=-1)
bagging_clf.fit(trainingDataFeatures, trainingDataLabels)
predictions = bagging_clf.predict(testDataFeatures)
accuracy_score(testDataLabels, predictions)

0.956140350877193

When I did this using a single decision tree classifier was 90.3% accurate, while the bagging classifier was 96.5% accurate--halving the error rate! that's pretty good!


### Pasting
Let's try the same thing with pasting (without replacement):

For that we set the hyperparameter: `bootstrap=False`


In [8]:
pasting_clf = BaggingClassifier(clf, n_estimators=20, max_samples=100, 
                                bootstrap=False, n_jobs=-1)
pasting_clf.fit(trainingDataFeatures, trainingDataLabels)
predictions = pasting_clf.predict(testDataFeatures)
accuracy_score(testDataLabels, predictions)

0.9385964912280702

### Random Subspaces
Again, random subspaces are when we randomly select feature subsets rather than subsets of the dataset instances. This time we will create 50 classifiers for our collective and each will train on a dataset with 7 features (`max_feature=7`).

In [9]:
subspace_clf = BaggingClassifier(clf, n_estimators=50, max_features=7, 
                                bootstrap=True, n_jobs=-1)
subspace_clf.fit(trainingDataFeatures, trainingDataLabels)
predictions = subspace_clf.predict(testDataFeatures)
accuracy_score(testDataLabels, predictions)

0.956140350877193

### Random Patches
Finally, let's combine things and try random patches. In this example each classifier will be given a subset of 100 training instances with 7 features each.:

In [10]:
subspace_clf = BaggingClassifier(clf, n_estimators=100, max_features=7, 
                                 max_samples=100, bootstrap=False, n_jobs=-1)
subspace_clf.fit(trainingDataFeatures, trainingDataLabels)
predictions = subspace_clf.predict(testDataFeatures)
accuracy_score(testDataLabels, predictions)

0.9385964912280702

While it is common to use a decision tree as the base classifier, we can use any classifier. Here we use kNN:

In [11]:
from sklearn.neighbors import KNeighborsClassifier
kNN = KNeighborsClassifier()
bagging_clf = BaggingClassifier(kNN, n_estimators=20, max_samples=100, 
                                bootstrap=True, n_jobs=-1)
bagging_clf.fit(trainingDataFeatures, trainingDataLabels)
predictions = bagging_clf.predict(testDataFeatures)
accuracy_score(testDataLabels, predictions)

0.9122807017543859

#### Summary
As you can see, any of these simple bagging algorithms typically outperforms using a single classifier. 


### Review

We import the bagging classifier library with:

```
from sklearn.ensemble import BaggingClassifier
```

and create an instance of one with:

```
my_bagging_classifier = BaggingClassifier(baseClassifier, Hyperparameters,n_jobs=-1)
```

#### Base Classifier
while any classifier can be used we typically use a decision tree

#### Hyperparameters
Here is a list of the hyperparameters (from the [sklearn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html#sklearn.ensemble.BaggingClassifier)):

* `n_estimators`: integer, default value = 10, the number of classifiers (estimators) in the ensemble.
* `max_samples`: integer or float, default value = 1.0(meaning use all the training instances), the number of samples (instances) to draw from the training dataset to train each base classifier.
    * if integer, then draw max_features features.
    * if float, then draw max_samples * X.shape[0] samples. For example if `max_samples` is 0.7 and there are 100 instances in the training dataset then draw 70 samples.
* `max_features`, integer or float, default value =1.0,  
the number of features to draw from the training dataset to train each base estimator 
    * if integers, then draw max_features features.
    * if float, then draw max_features * X.shape[1] features.
* `bootstrap` boolean, default value =True, whether samples and features are drawn with replacement. If False, sampling without replacement is performed.

For other hyperparamters, consult the documentation.


![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/torchdivide.png)


# <font color='#EE4C2C'>You Try ...</font> 
## <font color='#EE4C2C'>Predicting musical genres from audio file attributes</font> 


When you listen even to a few seconds of a song you can identify it as blues, country, classical, or any other genre. How do you do this? What attributes are you hearing in the audio file that helps you make this classification? And, more to the point, can we train a computer to do it?

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/bluesClassical.png)

We are going to be using  the [GTZAN Dataset for Music Genre Classification](https://www.kaggle.com/andradaolteanu/gtzan-dataset-music-genre-classification). It provides data of 100 songs for each of 10 genres. The data is in several formats:

* 30 second audio files (wav)
* spectral images of those 30 second clips (see image above)
* a csv file containing acoustic attributes of the 30 second clip
* a csv file containing acoustic attributes of 3 second clips (the 30 second clips were split into 3 second ones)

We are going to use the 3 second csv file which is available at 

https://raw.githubusercontent.com/zacharski/ml-class/master/data/gtzan.csv

Go ahead and load the data into a dataframe (the first row contains feature names)


In [34]:
music = pd.read_csv("https://raw.githubusercontent.com/zacharski/ml-class/master/data/gtzan.csv")
music.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues


Let's examine the values of the label column (the genres):

In [35]:
music.label.unique()

array(['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz',
       'metal', 'pop', 'reggae', 'rock'], dtype=object)

Those are the 10 genres we are trying to predict. So if we were just to guess without hearing the clip, we would be accurate 10% of the time. How accurate do you think you would be based on hearing a 3 second clip? I am pretty confident I could correctly label the 30 second clips, but I am much less confident about labeling 3 second ones. Since guessing randomly would give me 10% accuracy, I am estimating maybe 50-60% accuracy. Let's see how a computer does.

#### Feature Names
So the column we are trying to predict is `label`. Now let's get the names of the feature columns

In [36]:
featureNames = list(music.columns)
featureNames.remove('filename')
featureNames.remove('label')
print(featureNames)
print(len(featureNames))

['length', 'chroma_stft_mean', 'chroma_stft_var', 'rms_mean', 'rms_var', 'spectral_centroid_mean', 'spectral_centroid_var', 'spectral_bandwidth_mean', 'spectral_bandwidth_var', 'rolloff_mean', 'rolloff_var', 'zero_crossing_rate_mean', 'zero_crossing_rate_var', 'harmony_mean', 'harmony_var', 'perceptr_mean', 'perceptr_var', 'tempo', 'mfcc1_mean', 'mfcc1_var', 'mfcc2_mean', 'mfcc2_var', 'mfcc3_mean', 'mfcc3_var', 'mfcc4_mean', 'mfcc4_var', 'mfcc5_mean', 'mfcc5_var', 'mfcc6_mean', 'mfcc6_var', 'mfcc7_mean', 'mfcc7_var', 'mfcc8_mean', 'mfcc8_var', 'mfcc9_mean', 'mfcc9_var', 'mfcc10_mean', 'mfcc10_var', 'mfcc11_mean', 'mfcc11_var', 'mfcc12_mean', 'mfcc12_var', 'mfcc13_mean', 'mfcc13_var', 'mfcc14_mean', 'mfcc14_var', 'mfcc15_mean', 'mfcc15_var', 'mfcc16_mean', 'mfcc16_var', 'mfcc17_mean', 'mfcc17_var', 'mfcc18_mean', 'mfcc18_var', 'mfcc19_mean', 'mfcc19_var', 'mfcc20_mean', 'mfcc20_var']
58


In [41]:
df = music.drop(columns = ['filename'])
df

,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,3714.560359,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,3869.682242,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,3997.639160,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,3568.300218,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,3469.992864,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9985,66149,0.349126,0.080515,0.050019,0.000097,1499.083005,164266.886443,1718.707215,85931.574523,3015.559458,...,42.485981,-9.094270,38.326839,-4.246976,31.049839,-5.625813,48.804092,1.818823,38.966969,rock
9986,66149,0.372564,0.082626,0.057897,0.000088,1847.965128,281054.935973,1906.468492,99727.037054,3746.694524,...,32.415203,-12.375726,66.418587,-3.081278,54.414265,-11.960546,63.452255,0.428857,18.697033,rock
9987,66149,0.347481,0.089019,0.052403,0.000701,1346.157659,662956.246325,1561.859087,138762.841945,2442.362154,...,78.228149,-2.524483,21.778994,4.809936,25.980829,1.775686,48.582378,-0.299545,41.586990,rock
9988,66149,0.387527,0.084815,0.066430,0.000320,2084.515327,203891.039161,2018.366254,22860.992562,4313.266226,...,28.323744,-5.363541,17.209942,6.462601,21.442928,2.354765,24.843613,0.675824,12.787750,rock


So we have 58 features. 

#### Training and test sets
Now it is time to construct the training and test sets:

In [42]:
## divide the original data 80% going into the music_training dataset 
## the rest in music_test
music_training, music_test = train_test_split(df, train_size=0.80)
music_training                              


,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
2334,66149,0.319796,0.087282,0.135585,0.002536,2639.346930,334344.135600,2335.127138,84210.834199,4995.289025,...,52.081852,1.035279,75.083458,-5.938715,41.641727,-1.137613,59.863434,-1.235716,43.183453,country
677,66149,0.362836,0.086504,0.122606,0.000383,2101.160975,117284.602761,2162.885467,39101.359560,4598.167067,...,66.560081,-11.350907,45.567181,10.066656,49.160305,-3.161512,28.323057,-0.027508,48.727692,blues
8676,66149,0.352383,0.099683,0.102610,0.001936,2085.088054,819217.159889,2285.688571,164954.089465,4333.557129,...,48.379078,1.254592,91.048470,0.861770,82.595718,-0.676331,91.287636,-4.537113,47.836060,reggae
3031,66149,0.507811,0.074411,0.193821,0.002209,3105.599510,843800.509617,2837.234906,119217.521939,6487.705642,...,41.860165,-1.936218,45.630947,-3.601428,38.843704,-4.795041,34.939434,-0.574637,69.910469,disco
184,66149,0.369567,0.094109,0.051101,0.001135,1595.946934,212105.743165,2275.915548,161249.778242,4079.713792,...,58.479073,-4.580301,88.363190,6.640802,79.157425,7.630933,92.139053,3.720326,55.543072,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314,66149,0.281305,0.089618,0.025753,0.000153,928.678645,25483.322408,1241.905645,33254.126169,1664.102502,...,55.064411,-2.019715,68.745796,-2.364160,65.524475,0.401647,66.796623,0.381216,93.805534,classical
6262,66149,0.460453,0.081036,0.141595,0.000844,2215.339966,96745.761153,2218.316593,34206.252856,4800.247897,...,24.886000,-5.208145,39.541344,8.427248,36.150887,-5.771747,29.598787,0.090899,25.532009,metal
9700,66149,0.379821,0.089256,0.162020,0.001911,2020.277003,131395.112644,2250.266316,54722.117766,4706.412823,...,25.953970,-10.677932,38.016895,1.060117,77.836800,-5.780015,87.446533,4.848056,104.622589,rock
5765,66149,0.317982,0.082576,0.162828,0.001704,2486.492897,177082.963016,2496.233282,140634.452991,5170.370530,...,78.559494,-0.570864,79.854088,-4.018246,48.935555,-8.255075,55.127266,-11.049654,47.389420,jazz


In [44]:
## now create the DataFrames for just the features (excluding the label column 
## filename column)                              
music_training_features =  music_training.iloc[:,:-1]
music_test_features = music_test.iloc[:,:-1]
music_training_features                             

,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
2334,66149,0.319796,0.087282,0.135585,0.002536,2639.346930,334344.135600,2335.127138,84210.834199,4995.289025,...,-2.809328,52.081852,1.035279,75.083458,-5.938715,41.641727,-1.137613,59.863434,-1.235716,43.183453
677,66149,0.362836,0.086504,0.122606,0.000383,2101.160975,117284.602761,2162.885467,39101.359560,4598.167067,...,8.349647,66.560081,-11.350907,45.567181,10.066656,49.160305,-3.161512,28.323057,-0.027508,48.727692
8676,66149,0.352383,0.099683,0.102610,0.001936,2085.088054,819217.159889,2285.688571,164954.089465,4333.557129,...,6.846690,48.379078,1.254592,91.048470,0.861770,82.595718,-0.676331,91.287636,-4.537113,47.836060
3031,66149,0.507811,0.074411,0.193821,0.002209,3105.599510,843800.509617,2837.234906,119217.521939,6487.705642,...,1.244036,41.860165,-1.936218,45.630947,-3.601428,38.843704,-4.795041,34.939434,-0.574637,69.910469
184,66149,0.369567,0.094109,0.051101,0.001135,1595.946934,212105.743165,2275.915548,161249.778242,4079.713792,...,1.499707,58.479073,-4.580301,88.363190,6.640802,79.157425,7.630933,92.139053,3.720326,55.543072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314,66149,0.281305,0.089618,0.025753,0.000153,928.678645,25483.322408,1241.905645,33254.126169,1664.102502,...,0.262318,55.064411,-2.019715,68.745796,-2.364160,65.524475,0.401647,66.796623,0.381216,93.805534
6262,66149,0.460453,0.081036,0.141595,0.000844,2215.339966,96745.761153,2218.316593,34206.252856,4800.247897,...,4.683761,24.886000,-5.208145,39.541344,8.427248,36.150887,-5.771747,29.598787,0.090899,25.532009
9700,66149,0.379821,0.089256,0.162020,0.001911,2020.277003,131395.112644,2250.266316,54722.117766,4706.412823,...,5.192623,25.953970,-10.677932,38.016895,1.060117,77.836800,-5.780015,87.446533,4.848056,104.622589
5765,66149,0.317982,0.082576,0.162828,0.001704,2486.492897,177082.963016,2496.233282,140634.452991,5170.370530,...,-0.703714,78.559494,-0.570864,79.854088,-4.018246,48.935555,-8.255075,55.127266,-11.049654,47.389420


In [45]:
## now create the labels data structure for both the training and test sets                              
music_training_labels = music_training.iloc[:,-1]
music_test_labels = music_test.iloc[:,-1]
music_test_labels

3671        disco
1891    classical
6256        metal
1573    classical
3229        disco
          ...    
4033       hiphop
3335        disco
1110    classical
849         blues
1838    classical
Name: label, Length: 1998, dtype: object

### Building a single decision tree classifier
Let's build a single decision tree classifier called `clf` using entropy, fit it to the data, make predictions and determine the accuracy:

In [50]:
## Create clf, an instance of the Decision Tree Classifier
from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion='entropy')
## Fit it to the data
clf.fit(music_training_features, music_training_labels)
## get the predictions for the test set
predictions = clf.predict(music_test_features)
## get the accuracy score
accuracy_score(music_test_labels, predictions)

0.6701701701701702

When I did this I got 66% accuracy. That doesn't sound great but keep in mind that random guessing would only be 10% accuracy. 

### Building a Random Patch Classifier

Now we are going to build a random patch classifier.

* the base classifier will be a decision tree using entropy
* the ensemble will contain 20 base classifiers
* each classifier will use a random sample of 70% of the training data
* each classifier will use a random sample of 70% of the features
* the sampling will be done with replacement
* it will use all available cpu cores.

We are going to

1. build the classifier
2. train the classifier on the data
3. make predictions on the test set
4. determine the accuracy

In [51]:
subspace_clf = BaggingClassifier(clf, n_estimators=100, max_features=20, 
                                 max_samples=100, bootstrap=False, n_jobs=-1)
subspace_clf.fit(music_training_features, music_training_labels)
predictions1 = subspace_clf.predict(music_test_features)
accuracy_score(music_test_labels, predictions1)

0.6031031031031031

What accuracy did you get? Was it better than using a single classifier?
Keep your original code above. Make a copy of it below and 
experiment a bit with the hyperparameters. (try 3 or 4 different things) What is the best accuracy you can get?

#### Solution

we get the accuracy 60%, which is not bettter than the single classfier. 

In [53]:
from sklearn.neighbors import KNeighborsClassifier
kNN = KNeighborsClassifier()
bagging_clf = BaggingClassifier(kNN, n_estimators=200, max_samples=100, 
                                bootstrap=True, n_jobs=-1)
bagging_clf.fit(music_training_features, music_training_labels)
predictions2 = bagging_clf.predict(music_test_features)
accuracy_score(music_test_labels, predictions2)

0.2887887887887888


##<font color='#52BE80'>NOTES</font>

### Breast Cancer Database

[back](#Wisconsin-Cancer-Dataset)

  This breast cancer databases was obtained from the University of Wisconsin
   Hospitals, Madison from Dr. William H. Wolberg.  If you publish results
   when using this database, then please include this information in your
   acknowledgements.  Also, please cite one or more of:

   1. O. L. Mangasarian and W. H. Wolberg: "Cancer diagnosis via linear 
      programming", SIAM News, Volume 23, Number 5, September 1990, pp 1 & 18.

   2. William H. Wolberg and O.L. Mangasarian: "Multisurface method of 
      pattern separation for medical diagnosis applied to breast cytology", 
      Proceedings of the National Academy of Sciences, U.S.A., Volume 87, 
      December 1990, pp 9193-9196.

   3. O. L. Mangasarian, R. Setiono, and W.H. Wolberg: "Pattern recognition 
      via linear programming: Theory and application to medical diagnosis", 
      in: "Large-scale numerical optimization", Thomas F. Coleman and Yuying
      Li, editors, SIAM Publications, Philadelphia 1990, pp 22-30.

   4. K. P. Bennett & O. L. Mangasarian: "Robust linear programming 
      discrimination of two linearly inseparable sets", Optimization Methods
      and Software 1, 1992, 23-34 (Gordon & Breach Science Publishers).

Title: Wisconsin Breast Cancer Database (January 8, 1991)


Sources:
   -- Dr. WIlliam H. Wolberg (physician)
      University of Wisconsin Hospitals
      Madison, Wisconsin
      USA
   -- Donor: Olvi Mangasarian (mangasarian@cs.wisc.edu)
      Received by David W. Aha (aha@cs.jhu.edu)
   -- Date: 15 July 1992